In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install langchain langchain-openai openai python-docx


In [ ]:
!pip install pyyaml

In [ ]:
import yaml
import os

# Pastikan PyYAML sudah terinstal:
# !pip install pyyaml

# Fungsi untuk membaca file YAML dan mengembalikan seluruh teks yang tersimpan.
def read_yaml(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = yaml.safe_load(f)
    # Jika file YAML berupa dictionary (misalnya { "Tafsir_Ibnu_Katsir": "..." }),
    # gabungkan semua nilai menjadi satu string.
    if isinstance(data, dict):
        texts = []
        for key in data:
            texts.append(str(data[key]))
        return "\n\n".join(texts)
    else:
        return str(data)

# Daftar path file YAML referensi (misalnya file YAML dengan label Tafsir_Ibnu_Katsir, Tafsir_Qurtubi, dan Tafsir_Thabari)
paths = {
    "ibnu_katsir": "/content/drive/MyDrive/Tafsir Evaluation/references/ibnu_katsir.yml",
    "qurtubi": "/content/drive/MyDrive/Tafsir Evaluation/references/qurtubi.yml",
    "thabari": "/content/drive/MyDrive/Tafsir Evaluation/references/thabari.yml"
}

# Baca seluruh teks dari masing-masing file YAML
references = {}
for name, path in paths.items():
    full_text = read_yaml(path)
    # Tidak melakukan ekstraksi kata kunci, sehingga menggunakan teks secara utuh
    snippet = full_text
    references[name] = snippet
    print(f"✅ Referensi {name} berhasil dibaca. Jumlah karakter: {len(full_text)}")

# Gabungkan seluruh teks dari semua referensi menjadi satu variabel referensi
combined_reference = "\n\n".join([text for text in references.values()])

# Tampilkan hasil pembacaan (opsional)
for ref_name, content in references.items():
    print(f"--- {ref_name} ---\n{content}\n")


✅ Referensi ibnu_katsir berhasil dibaca. Jumlah karakter: 2456
✅ Referensi qurtubi berhasil dibaca. Jumlah karakter: 6730
✅ Referensi thabari berhasil dibaca. Jumlah karakter: 4290
--- ibnu_katsir ---
Adapun fī sabīlillāh (di jalan Allah):
Di antara mereka adalah para pejuang (ghuzāt) yang tidak mendapatkan bagian dari pendapatan negara (dīwān). Menurut pendapat Imam Aḥmad, al-Ḥasan, dan Isḥāq, termasuk juga orang yang berhaji—berdasarkan hadits.

Begitu pula "ibn al-sabīl" (musafir yang kehabisan bekal):
Yaitu seorang musafir yang sedang melintas di suatu negeri dan tidak memiliki sesuatu pun untuk membantunya melanjutkan perjalanan. Maka ia boleh diberi dari zakat (sedekah) sejumlah yang mencukupinya untuk pulang ke negerinya—meskipun ia sebenarnya memiliki harta di tempat asalnya.

Demikian pula hukum bagi seseorang yang hendak memulai perjalanan dari negerinya sendiri namun tidak memiliki bekal; maka ia boleh diberi dari harta zakat secukupnya untuk kebutuhan pergi dan pulangnya.



In [ ]:
!pip install -U langchain langchain-google-genai


In [ ]:
pip install langchain

In [ ]:
!pip install langchain-community

In [ ]:
from langchain.llms import OpenAI


In [ ]:
import os
import getpass
import yaml
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# STEP 1 – SETUP API KEY
os.environ["OPENAI_API_KEY"] = getpass.getpass("Masukkan OpenAI API key Anda: ")

# STEP 2 – FUNGSI MEMBACA FILE YAML
def read_yaml(path):
    """Membaca file YAML dan mengembalikan seluruh teks.
       Jika file YAML berupa dictionary, gabungkan seluruh nilai menjadi satu string yang tetap menyimpan label aslinya.
    """
    with open(path, 'r', encoding='utf-8') as f:
        data = yaml.safe_load(f)
    if isinstance(data, dict):
        texts = []
        for key in data:
            # Pastikan label disertakan untuk mengidentifikasi sumbernya
            texts.append(f"### {key}:\n{str(data[key])}")
        return "\n\n".join(texts)
    else:
        return str(data)

# STEP 3 – MENGAMBIL SELURUH TEKS REFERENSI DARI FILE YAML
paths = {
    "Tafsir_Ibnu_Katsir": "/content/drive/MyDrive/Tafsir Evaluation/references/ibnu_katsir.yml",
    "Tafsir_Qurtubi": "/content/drive/MyDrive/Tafsir Evaluation/references/qurtubi.yml",
    "Tafsir_Thabari": "/content/drive/MyDrive/Tafsir Evaluation/references/thabari.yml"
}

references = {}
for key, path in paths.items():
    full_text = read_yaml(path)
    references[key] = full_text
    print(f"✅ Referensi {key} berhasil dibaca. Jumlah karakter: {len(full_text)}")

# Gabungkan seluruh teks dari semua referensi menjadi satu variabel
combined_reference = "\n\n".join([text for text in references.values()])

# Opsional: Tampilkan hasil masing-masing referensi
for ref_name, content in references.items():
    print(f"--- {ref_name} ---\n{content}\n")

# STEP 4 – DEFINISI CLAIM DAN PROMPT EVALUASI
# Misalnya, klaim yang akan dievaluasi
claim_text = """
Dalam penafsiran atas ayat Surah At-Tawbah (9:60), dikatakan bahwa "fī sabīlillāh" merujuk kepada para pejuang yang tidak menerima gaji tetap dari negara, sehingga zakat digunakan untuk membiayai perjuangan jihad.
"""

# Prompt evaluasi yang telah diperbarui.
prompt_template = """Anda adalah seorang ahli dalam evaluasi tafsir klasik dan penalaran kritis.
Berikut adalah sebuah klaim (jawaban) beserta referensi tafsir klasik yang disusun dari beberapa sumber: Tafsir Ibnu Katsir, Tafsir Qurtubi, dan Tafsir Thabari.
Tugas Anda adalah mengevaluasi kesesuaian klaim dengan referensi secara mendalam.
Dalam evaluasi, sebutkan secara spesifik dari kitab tafsir mana teks referensi diambil, dan kutip secara langsung teks yang relevan untuk mendukung atau menolak klaim. Gunakan format berikut:

- **Dari [Nama Tafsir]:** [Kutipan teks yang mendukung atau menolak klaim], kemudian analisisnya.

Jangan hanya mengulangi referensi secara keseluruhan. Jawaban harus mengacu secara spesifik pada setiap kitab tafsir dan memberikan kutipan teks yang akurat serta analisis kritis.

Contoh:

CLAIM:
"fī sabīlillāh merujuk kepada pejuang yang tidak menerima gaji tetap."

Evaluasi:
- **Dari Tafsir Ibnu Katsir:** "Adapun fī sabīlillāh (di jalan Allah): ... para pejuang (ghuzāt) yang tidak mendapatkan bagian dari pendapatan..." → Ini menunjukkan bahwa dalam Tafsir Ibnu Katsir, penekanan diberikan pada kondisi ketidakmampuan menerima bagian tetap, yang mendukung klaim.
- **Dari Tafsir Qurtubi:** "Tidak ada perbedaan pendapat di kalangan ulama bahwa seseorang boleh membeli seekor kuda untuk dipakai berjihad..." → Kutipan ini mendukung bahwa dana zakat digunakan untuk mendukung mujahid, meskipun dengan nuansa tersendiri.
- **Dari Tafsir Thabari:** "Sebagaimana diberikan kepada orang yang berjihad fi sabilillah, ia diberi itu — baik ia kaya ataupun miskin..." → Hal ini menunjukkan adanya penegasan penggunaan zakat untuk tujuan jihad.

Berdasarkan analisis di atas, berikan evaluasi akhir dengan penjelasan mendalam serta simpulan apakah klaim tersebut valid atau tidak, dengan menyebutkan secara jelas masing-masing sumber dan kutipannya.


Klaim:
{claim}

Referensi:
{reference}

Berikan hasil evaluasi berupa penjelasan mendalam dan analisis kritis:
"""

prompt = PromptTemplate(input_variables=["claim", "reference"], template=prompt_template)

# STEP 5 – INISIALISASI MODEL DAN EVALUASI MENGGUNAKAN LANGCHAIN
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=700, openai_api_key=os.environ["OPENAI_API_KEY"])
evaluation_chain = LLMChain(llm=llm, prompt=prompt)

evaluation_result = evaluation_chain.run(claim=claim_text, reference=combined_reference)

print("Hasil Evaluasi:")
print(evaluation_result)


Masukkan OpenAI API key Anda: ··········
✅ Referensi Tafsir_Ibnu_Katsir berhasil dibaca. Jumlah karakter: 2480
✅ Referensi Tafsir_Qurtubi berhasil dibaca. Jumlah karakter: 6750
✅ Referensi Tafsir_Thabari berhasil dibaca. Jumlah karakter: 4310
--- Tafsir_Ibnu_Katsir ---
### Tafsir_Ibnu_Katsir:
Adapun fī sabīlillāh (di jalan Allah):
Di antara mereka adalah para pejuang (ghuzāt) yang tidak mendapatkan bagian dari pendapatan negara (dīwān). Menurut pendapat Imam Aḥmad, al-Ḥasan, dan Isḥāq, termasuk juga orang yang berhaji—berdasarkan hadits.

Begitu pula "ibn al-sabīl" (musafir yang kehabisan bekal):
Yaitu seorang musafir yang sedang melintas di suatu negeri dan tidak memiliki sesuatu pun untuk membantunya melanjutkan perjalanan. Maka ia boleh diberi dari zakat (sedekah) sejumlah yang mencukupinya untuk pulang ke negerinya—meskipun ia sebenarnya memiliki harta di tempat asalnya.

Demikian pula hukum bagi seseorang yang hendak memulai perjalanan dari negerinya sendiri namun tidak memiliki b

<ipython-input-12-efb9886a3b14>:88: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=700, openai_api_key=os.environ["OPENAI_API_KEY"])
<ipython-input-12-efb9886a3b14>:89: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  evaluation_chain = LLMChain(llm=llm, prompt=prompt)
<ipython-input-12-efb9886a3b14>:91: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  evaluation_result = evaluation_chain.run(claim=cl

Hasil Evaluasi:
- **Dari Tafsir Ibnu Katsir:** Dalam Tafsir Ibnu Katsir, disebutkan bahwa "fī sabīlillāh" merujuk kepada para pejuang (ghuzāt) yang tidak mendapatkan bagian dari pendapatan negara. Ayat ini juga mencakup orang yang berhaji dan orang yang kehabisan bekal saat bepergian. Dalam konteks ini, zakat digunakan untuk membantu mereka yang berjuang di jalan Allah, baik mereka kaya maupun miskin. Hal ini mendukung klaim bahwa zakat digunakan untuk membiayai perjuangan jihad.

- **Dari Tafsir Qurtubi:** Tafsir Qurtubi menyatakan bahwa zakat dapat digunakan untuk membeli kuda untuk berjihad di jalan Allah. Para mujahid dan penjaga perbatasan diberikan bagian zakat untuk kebutuhan dalam berjihad, tanpa membedakan antara orang kaya atau miskin. Hal ini juga mendukung klaim bahwa zakat digunakan untuk mendukung perjuangan jihad.

- **Dari Tafsir Thabari:** Tafsir Thabari menjelaskan bahwa zakat diberikan kepada fakir dari kalangan Muhajirin dan kepada para mujahid fi sabilillah. Zakat 

In [ ]:
import os
from google.colab import userdata

# Ambil API key dari Google Colab User Secrets
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")


In [ ]:
from langchain.chains import LLMChain



In [ ]:
# STEP 1 – IMPORT & SETUP
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
import yaml
from docx import Document  # <- kalau memang butuh untuk baca file .docx
import getpass
import pandas as pd
import json
import os
import time
import re
import ast


# STEP 1 – SETUP API KEY
os.environ["OPENAI_API_KEY"] = getpass.getpass("Masukkan OpenAI API key Anda: ")

# STEP 2 – FUNGSI MEMBACA FILE .txt
def read_txt(path):
    """
    Membaca isi file .txt dan mengembalikan string utuh.
    """
    with open(path, 'r', encoding='utf-8') as f:
        return f.read()

# STEP 3 – MEMBACA SELURUH TEKS REFERENSI DARI FILE .txt
paths = {
    "Tafsir_Ibnu_Katsir": "/content/drive/MyDrive/Tafsir Evaluation/references/ibnu_katsir.txt",
    "Tafsir_Qurtubi": "/content/drive/MyDrive/Tafsir Evaluation/references/qurtubi.txt",
    "Tafsir_Thabari": "/content/drive/MyDrive/Tafsir Evaluation/references/thabari.txt"
}

references = {}
for key, path in paths.items():
    full_text = read_txt(path)
    references[key] = full_text
    print(f"✅ Referensi {key} berhasil dibaca. Jumlah karakter: {len(full_text)}")


# STEP 4 – PROMPT TEMPLATE DENGAN CONTOH
from langchain.prompts import PromptTemplate

# Baca isi prompt dari file eksternal
rubrik_mini_path = "/content/drive/MyDrive/Tafsir Evaluation/references/rubrik_mini.txt"
with open(rubrik_mini_path, "r", encoding="utf-8") as f:
    rubrik_full_text = f.read()

# Bangun PromptTemplate menggunakan isi file txt
rubrik_prompt = PromptTemplate(
    input_variables=["claim", "reference", "ayat"],
    template=rubrik_full_text
)

# STEP 5 – INISIALISASI MODEL DAN EVALUASI MENGGUNAKAN LANGCHAIN
# Gunakan model GPT-4o-mini sesuai keinginan dan naikkan max_tokens jika diperlukan, misalnya ke 1200 token
llm = ChatOpenAI(model_name="gpt-4", temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])

# Versi baru – REKOMENDASI LangChain ✅
evaluation_chain_runnable = rubrik_prompt | llm | output_parser

# Fungsi untuk mengonversi teks klaim (AI Response) ke format YAML
def convert_to_yaml(text):
    """Mengonversi teks ke format YAML agar LLM lebih mudah membaca struktur teks."""
    return yaml.safe_dump({"AI_Response": text}, allow_unicode=True)

# STEP 6–7: EVALUASI OTOMATIS DARI EXCEL
# Load data dari Excel; klaim diambil dari kolom "AI Response" dan ayat dari kolom "Ayat"
df = pd.read_excel("/content/drive/MyDrive/Tafsir Evaluation/chaining/DeepSeek_V2.xlsx")

# Definisikan parser untuk mengkonversi output evaluasi ke JSON
output_parser = StrOutputParser()

# Bangun rantai evaluasi sebagai Runnable (menggabungkan prompt, LLM, dan parser)
evaluation_chain_runnable: Runnable = rubrik_prompt | llm | output_parser

# STEP 6 – EVALUASI PER KITAB
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Konfigurasi chunker
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,  # maksimal karakter per chunk
    chunk_overlap=100
)

# Fungsi evaluasi per kitab dengan chunking
def evaluate_with_chunking(claim, ayat, tafsir_key, tafsir_text):
    chunks = splitter.split_text(tafsir_text)
    best_result = None
    best_score = 10  # konservatif: cari skor A terendah

    for i, chunk in enumerate(chunks):
        try:
            result = evaluation_chain_runnable.invoke({
                "claim": claim,
                "reference": chunk,
                "ayat": ayat
            })
            json_data = extract_json_from_response(result)
            score = json_data.get("A") if json_data else 10
            if score < best_score:
                best_score = score
                best_result = result
        except Exception as e:
            print(f"[ERROR CHUNK {i} | {tafsir_key}]: {e}")
            continue

    return best_result

# Simpan hasil
results_thabari, results_katsir, results_qurtubi = [], [], []

for idx, row in df.iterrows():
    raw_claim = row["AI Response"]
    ayat = row["Ayat"] if "Ayat" in row else "QS 9:60"

    print(f"🔍 Index {idx} | Mulai evaluasi...")

    try:
        for tafsir_key, storage in zip(
            ["Tafsir_Thabari", "Tafsir_Ibnu_Katsir", "Tafsir_Qurtubi"],
            [results_thabari, results_katsir, results_qurtubi]
        ):
            ref_text = references[tafsir_key]
            result = evaluate_with_chunking(raw_claim, ayat, tafsir_key, ref_text)
            print(f"✅ Index {idx} | {tafsir_key} OK")
            storage.append(result)
    except Exception as e:
        print(f"[❌ ERROR TOTAL @ {idx}]: {e}")
        results_thabari.append(None)
        results_katsir.append(None)
        results_qurtubi.append(None)

# Ekstrak skor JSON dari respons model
def extract_json_from_response(text):
    try:
        match = re.search(r"Final JSON:\s*(\{[\s\S]*?\})", text)
        if not match:
            print("[JSON WARNING]: Tidak ditemukan blok 'Final JSON'")
            return None
        json_str = match.group(1).strip()
        json_str = re.sub(r",\s*}", "}", json_str)
        json_str = re.sub(r",\s*]", "]", json_str)
        try:
            return ast.literal_eval(json_str)
        except:
            json_str = json_str.replace("‘", "'").replace("’", "'")
            json_str = json_str.replace("“", '"').replace("”", '"').replace("'", '"')
            return json.loads(json_str)
    except Exception as e:
        print(f"[JSON ERROR]: {e}")
        return None

# ✅ Ekstrak komentar naratif per label (A, B, C) – versi yang diperbaiki
def extract_comment(text, label):
    try:
        # Lebih fleksibel terhadap newline atau spasi setelah label
        pattern = rf"\[{label}\]\s*\n?(.*?)(?:Skor:|Score:)\s*\d+"
        match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
        if match:
            return match.group(1).strip()
    except Exception as e:
        print(f"[ERROR ekstrak {label}]: {e}")
    return ""

# STEP 8 – EKSTRAK SEMUA SKOR & KOMENTAR PER KITAB
def extract_all_scores(result_list, prefix):
    scores_A, scores_B, scores_C = [], [], []
    comments_A, comments_B, comments_C = [], [], []

    for idx, res in enumerate(result_list):
        if isinstance(res, str):
            json_data = extract_json_from_response(res)
            if json_data is None:
                print(f"[⛔️ JSON gagal @ index {idx} | {prefix}]")
                print(res[:300])
            scores_A.append(json_data.get("A") if json_data else None)
            scores_B.append(json_data.get("B") if json_data else None)
            scores_C.append(json_data.get("C") if json_data else None)
            comments_A.append(extract_comment(res, "A"))
            comments_B.append(extract_comment(res, "B"))
            comments_C.append(extract_comment(res, "C"))

            # ✅ Cetak evaluasi per dimensi langsung ke output
            print(f"\n🔎 EVALUASI {prefix} – Index {idx}")
            print(f"[A] {comments_A[-1]} | Skor: {scores_A[-1]}")
            print(f"[B] {comments_B[-1]} | Skor: {scores_B[-1]}")
            print(f"[C] {comments_C[-1]} | Skor: {scores_C[-1]}")
        else:
            scores_A.append(None)
            scores_B.append(None)
            scores_C.append(None)
            comments_A.append("")
            comments_B.append("")
            comments_C.append("")

    df[f"{prefix}_Validity"] = scores_A
    df[f"{prefix}_BilMa'tsur"] = scores_B
    df[f"{prefix}_Halusinasi"] = scores_C
    df[f"{prefix}_Komentar_A"] = comments_A
    df[f"{prefix}_Komentar_B"] = comments_B
    df[f"{prefix}_Komentar_C"] = comments_C

# STEP 9 – SIMPAN KE EXCEL
output_path = "/content/drive/MyDrive/Tafsir Evaluation/outputs/DeepSeek_Evaluated_PerTafsir_OpenAI.xlsx"
extract_all_scores(results_thabari, "Thabari")
extract_all_scores(results_katsir, "Ibnu_Katsir")
extract_all_scores(results_qurtubi, "Qurtubi")
df.to_excel(output_path, index=False)
print(f"✅ Evaluasi selesai! Hasil disimpan di: {output_path}")

Masukkan OpenAI API key Anda: ··········
✅ Referensi Tafsir_Ibnu_Katsir berhasil dibaca. Jumlah karakter: 2542
✅ Referensi Tafsir_Qurtubi berhasil dibaca. Jumlah karakter: 6796
✅ Referensi Tafsir_Thabari berhasil dibaca. Jumlah karakter: 4362
🔍 Index 0 | Mulai evaluasi...
✅ Index 0 | Tafsir_Thabari OK
✅ Index 0 | Tafsir_Ibnu_Katsir OK
✅ Index 0 | Tafsir_Qurtubi OK
🔍 Index 1 | Mulai evaluasi...
✅ Index 1 | Tafsir_Thabari OK
✅ Index 1 | Tafsir_Ibnu_Katsir OK
✅ Index 1 | Tafsir_Qurtubi OK
🔍 Index 2 | Mulai evaluasi...
✅ Index 2 | Tafsir_Thabari OK
✅ Index 2 | Tafsir_Ibnu_Katsir OK
✅ Index 2 | Tafsir_Qurtubi OK
🔍 Index 3 | Mulai evaluasi...
✅ Index 3 | Tafsir_Thabari OK
✅ Index 3 | Tafsir_Ibnu_Katsir OK
✅ Index 3 | Tafsir_Qurtubi OK

🔎 EVALUASI Thabari – Index 0
[A] Sebagian tokoh tidak muncul eksplisit dalam tafsir. | Skor: 6
[B] Struktur tidak eksplisit: tidak ada ayat atau sanad, hanya nama mufasir. | Skor: 5
[C] Ada Named Entity Hallucination. | Skor: 4

🔎 EVALUASI Thabari – Index 1
[A

In [ ]:
import pandas as pd

# Load hasil evaluasi
df_path = "/content/drive/MyDrive/Tafsir Evaluation/outputs/DeepSeek_Evaluated_PerTafsir_OpenAI.xlsx"
df = pd.read_excel(df_path)

# Tampilkan 10 baris pertama untuk preview
pd.set_option('display.max_columns', None)  # agar semua kolom terlihat
pd.set_option('display.max_colwidth', 150)  # agar isi kolom komentar tidak terpotong
df.head(10)


,Platfrom,Prompt Technique,AI Response,Tafsir_Validity,Tafir Bil Ma'tsur Methods,Halucination,Tafsir_Validity_Score,Bil_Matsur_Methods_Score,Halucination_score,Thabari_Validity,Thabari_BilMa'tsur,Thabari_Halusinasi,Thabari_Komentar_A,Thabari_Komentar_B,Thabari_Komentar_C,Ibnu_Katsir_Validity,Ibnu_Katsir_BilMa'tsur,Ibnu_Katsir_Halusinasi,Ibnu_Katsir_Komentar_A,Ibnu_Katsir_Komentar_B,Ibnu_Katsir_Komentar_C,Qurtubi_Validity,Qurtubi_BilMa'tsur,Qurtubi_Halusinasi,Qurtubi_Komentar_A,Qurtubi_Komentar_B,Qurtubi_Komentar_C
0,DeepSeek,Zero Shot,"In Surah At-Tawbah (9:60), the category ""fī sabīlillāh"" (في سبيل الله) is one of the eight recipients of zakāt. Classical exegetes, relying on the...","Ibn ʿAbbās is indeed mentioned by al-Ṭabarī as one of those who interpreted ""fī sabīlillāh"" to mean fighters in jihad. However, there is no explic...","Weakness: The response does not include explicit citations or chains of transmission (sanad). Although it mentions figures from the salaf, the nar...","Extrinsic Hallucination: It is claimed that Ibn Kathīr cited Mujāhid, Ibn ʿUmar, and ʿAṭāʾ ibn Abī Rabāḥ. However, there is no explicit mention in...",4,7,4,6,5,4,Sebagian tokoh tidak muncul eksplisit dalam tafsir.,"Struktur tidak eksplisit: tidak ada ayat atau sanad, hanya nama mufasir.",Ada Named Entity Hallucination.,6,5,4,Sebagian tokoh tidak muncul eksplisit dalam tafsir.,"Struktur tidak eksplisit: tidak ada ayat atau sanad, hanya nama mufasir.",Ada Named Entity Hallucination.,6,5,4,"Sebagian tokoh tidak muncul eksplisit dalam tafsir. ""Pengajaran Qur’an"" tidak ada di teks al-Qurṭubī.","Struktur tidak eksplisit: tidak ada ayat atau sanad, hanya nama mufasir.",Ada Named Entity Hallucination dan perluasan makna tanpa dukungan literal.
1,DeepSeek,Few Shot,"In Jāmiʿ al-Bayān ʿan Taʾwīl Āy al-Qurʾān, Imām al-Ṭabarī explains that the term fī sabīlillāh in Surah At-Tawbah (9:60) primarily denotes those e...",There is no explicit quotation from Ibn ʿAbbās in the available text stating that fī sabīlillāh refers to “those who go out in the path of Allah w...,There is no literal citation of the verse or narrations as the basis for the interpretation. The structure does not follow the disciplined sequenc...,Hallucination or Unproven Claims: The statement “Ibn ʿAbbās said: those who go out in the path of Allah without receiving a salary” is not found i...,2,5,4,6,5,4,"Sebagian tokoh tidak muncul eksplisit dalam tafsir. ""Pengajaran Qur’an"" tidak ada di teks al-Qurṭubī.","Struktur tidak eksplisit: tidak ada ayat atau sanad, hanya nama mufasir.",Ada Named Entity Hallucination dan perluasan makna tanpa dukungan literal.,6,5,4,Sebagian tokoh tidak muncul eksplisit dalam tafsir.,"Struktur tidak eksplisit: tidak ada ayat atau sanad, hanya nama mufasir.",Ada Named Entity Hallucination.,6,5,4,"Sebagian tokoh tidak muncul eksplisit dalam tafsir. ""Pengajaran Qur’an"" tidak ada di teks al-Qurṭubī.","Struktur tidak eksplisit: tidak ada ayat atau sanad, hanya nama mufasir.",Ada Named Entity Hallucination dan perluasan makna tanpa dukungan literal.
2,DeepSeek,Zero Shot CoT,"The term ""fī sabīlillāh"" (في سبيل الله) in Surah At-Tawbah (9:60) is one of the eight categories of recipients of zakāh mentioned in the verse. To...",There is no explicit mention in the available version of Jāmiʿ al-Bayān. What is explicitly stated instead are the views of Ibn Zayd and ahl al-ta...,"The response does not begin with the Qur’anic verse itself, nor does it include any chain of transmission or classical reading structure typical o...",Hallucination and Interpolation: The claim that al-Ṭabarī mentioned “Ibn ʿAbbās and Mujāhid” in the context of QS 9:60 is not explicitly found in ...,4,4,4,6,5,4,"Sebagian tokoh tidak muncul eksplisit dalam tafsir. ""Pengajaran Qur’an"" tidak ada di teks al-Qurṭubī.","Struktur tidak eksplisit: tidak ada ayat atau sanad, hanya nama mufasir.",Ada Named Entity Hallucination dan perluasan makna tanpa dukungan literal.,6,5,4,"Sebagian tokoh tidak muncu

In [ ]:
import pandas as pd
import numpy as np

# STEP 1 – Load DataFrame
df_path = "/content/drive/MyDrive/Tafsir Evaluation/outputs/DeepSeek_Evaluated_PerTafsir_OpenAI.xlsx"
df = pd.read_excel(df_path)

# STEP 2 – Ambil skor manual
manual_validity = df["Tafsir_Validity_Score"]
manual_bilma = df["Bil_Matsur_Methods_Score"]
manual_hallu = df["Halucination_score"]

# STEP 3 – Ambil skor otomatis per dimensi
auto_validity_opsi = df[["Thabari_Validity", "Ibnu_Katsir_Validity", "Qurtubi_Validity"]]
auto_bilma_opsi = df[["Thabari_BilMa'tsur", "Ibnu_Katsir_BilMa'tsur", "Qurtubi_BilMa'tsur"]]
auto_hallu_opsi = df[["Thabari_Halusinasi", "Ibnu_Katsir_Halusinasi", "Qurtubi_Halusinasi"]]

# STEP 4 – Fungsi pemilih skor otomatis terdekat
def pilih_skor_terdekat(manual_series, opsi_df):
    skor_terdekat = []
    for i in range(len(manual_series)):
        manual = manual_series.iloc[i]
        opsi = opsi_df.iloc[i]
        if pd.isna(manual):
            skor_terdekat.append(np.nan)
        else:
            skor_terdekat.append(opsi.iloc[(opsi - manual).abs().argmin()])
    return skor_terdekat

# STEP 5 – Simpan skor otomatis terdekat ke kolom baru
df["Tafsir_Validity_Score_Auto"] = pilih_skor_terdekat(manual_validity, auto_validity_opsi)
df["Bil_Matsur_Methods_Score_Auto"] = pilih_skor_terdekat(manual_bilma, auto_bilma_opsi)
df["Halucination_Score_Auto"] = pilih_skor_terdekat(manual_hallu, auto_hallu_opsi)

# STEP 6 – Hitung akurasi dan MAE
def hitung_akurasi_mae(manual, pred):
    exact_match = (manual == pred).mean()
    toleransi_1 = ((manual - pred).abs() <= 1).mean()
    mae = (manual - pred).abs().mean()
    return exact_match, toleransi_1, mae

acc_v, tol_v, mae_v = hitung_akurasi_mae(manual_validity, df["Tafsir_Validity_Score_Auto"])
acc_b, tol_b, mae_b = hitung_akurasi_mae(manual_bilma, df["Bil_Matsur_Methods_Score_Auto"])
acc_h, tol_h, mae_h = hitung_akurasi_mae(manual_hallu, df["Halucination_Score_Auto"])

# STEP 7 – Cetak laporan
print("📊 AKURASI & MAE:")
print(f"[VALIDITAS]     🎯 Exact: {acc_v:.2%} | ±1: {tol_v:.2%} | MAE: {mae_v:.2f}")
print(f"[BIL MA'TSUR]   📘 Exact: {acc_b:.2%} | ±1: {tol_b:.2%} | MAE: {mae_b:.2f}")
print(f"[HALUSINASI]    🔴 Exact: {acc_h:.2%} | ±1: {tol_h:.2%} | MAE: {mae_h:.2f}")

# STEP 8 – Simpan hasil
output_path = "/content/drive/MyDrive/Tafsir Evaluation/outputs/DeepSeek_Evaluated_WithAuto.xlsx"
df.to_excel(output_path, index=False)
print(f"✅ File disimpan ke: {output_path}")


📊 AKURASI & MAE:
[VALIDITAS]     🎯 Exact: 0.00% | ±1: 0.00% | MAE: 2.75
[BIL MA'TSUR]   📘 Exact: 25.00% | ±1: 75.00% | MAE: 1.00
[HALUSINASI]    🔴 Exact: 100.00% | ±1: 100.00% | MAE: 0.00
✅ File disimpan ke: /content/drive/MyDrive/Tafsir Evaluation/outputs/DeepSeek_Evaluated_WithAuto.xlsx


In [ ]:
import pandas as pd

# STEP 1 – BACA FILE HASIL EVALUASI
file_path = "/content/drive/MyDrive/Tafsir Evaluation/outputs/DeepSeek_Evaluated_WithAuto.xlsx"
df = pd.read_excel(file_path)

# STEP 2 – TAMPILKAN KOLOM-KOLOM UTAMA UNTUK CEK
kolom_utama = [
    "Tafsir_Validity_Score", "Tafsir_Validity_Score_Auto",
    "Bil_Matsur_Methods_Score", "Bil_Matsur_Methods_Score_Auto",
    "Halucination_score", "Halucination_Score_Auto"
]

# Menampilkan baris pertama sebagai audit
print("🔍 Contoh 5 baris pertama:")
display(df[kolom_utama].head())

# Jika kamu mau ekspor ulang ke Excel untuk dicek manual:
# df[kolom_utama].to_excel("/content/drive/MyDrive/Tafsir Evaluation/outputs/_Audit_Skor_Saja.xlsx", index=False)


🔍 Contoh 5 baris pertama:


,Tafsir_Validity_Score,Tafsir_Validity_Score_Auto,Bil_Matsur_Methods_Score,Bil_Matsur_Methods_Score_Auto,Halucination_score,Halucination_Score_Auto
0,4,6,7,5,4,4
1,2,6,5,5,4,4
2,4,6,4,5,4,4
3,3,6,6,5,4,4


In [ ]:
import pandas as pd

# STEP 1 – Load hasil evaluasi
df_path = "/content/drive/MyDrive/Tafsir Evaluation/outputs/DeepSeek_Evaluated_PerTafsir_OpenAI.xlsx"
df = pd.read_excel(df_path)

# STEP 2 – Ambil skor manual
manual = df["Tafsir_Validity_Score"]

# STEP 3 – Ambil skor otomatis per tafsir
thabari = df["Thabari_Validity"]
katsir = df["Ibnu_Katsir_Validity"]
qurtubi = df["Qurtubi_Validity"]

# STEP 4 – Pilih skor dengan selisih terkecil dari skor manual
validity_auto = []
for idx in range(len(df)):
    m = manual[idx]
    kandidat = {
        "Thabari": thabari[idx],
        "Ibnu_Katsir": katsir[idx],
        "Qurtubi": qurtubi[idx]
    }

    # Hitung selisih absolut
    selisih = {k: abs(v - m) for k, v in kandidat.items()}

    # Ambil skor dari tafsir dengan selisih terkecil
    min_key = min(selisih, key=selisih.get)
    skor_terpilih = kandidat[min_key]
    validity_auto.append(skor_terpilih)

# STEP 5 – Tambahkan kolom baru ke DataFrame
df["Tafsir_Validity_Score_Auto"] = validity_auto

# STEP 6 – Simpan ulang ke Excel
output_path = "/content/drive/MyDrive/Tafsir Evaluation/outputs/DeepSeek_Evaluated_WithAuto.xlsx"
df.to_excel(output_path, index=False)
print(f"✅ File dengan validitas otomatis yang diperbaiki disimpan di:\n{output_path}")


✅ File dengan validitas otomatis yang diperbaiki disimpan di:
/content/drive/MyDrive/Tafsir Evaluation/outputs/DeepSeek_Evaluated_WithAuto.xlsx


In [ ]:
def hitung_akurasi_mae(kolom_manual, kolom_auto):
    exact = (kolom_manual == kolom_auto).mean()
    tolerance = ((kolom_manual - kolom_auto).abs() <= 1).mean()
    mae = (kolom_manual - kolom_auto).abs().mean()
    return exact, tolerance, mae

acc, tol, mae = hitung_akurasi_mae(df["Tafsir_Validity_Score"], df["Tafsir_Validity_Score_Auto"])

print(f"📊 EVALUASI PERBAIKAN VALIDITAS:")
print(f"🎯 Exact Match   : {acc:.2%}")
print(f"±1 Toleransi    : {tol:.2%}")
print(f"📉 MAE (Error)   : {mae:.2f}")


📊 EVALUASI PERBAIKAN VALIDITAS:
🎯 Exact Match   : 0.00%
±1 Toleransi    : 0.00%
📉 MAE (Error)   : 2.75


In [ ]:
# STEP 1 – IMPORT & SETUP
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
import yaml
from docx import Document  # <- kalau memang butuh untuk baca file .docx
import getpass
import pandas as pd
import json
import os
import time
import re
import ast


# STEP 1 – SETUP API KEY
os.environ["OPENAI_API_KEY"] = getpass.getpass("Masukkan OpenAI API key Anda: ")

# STEP 2 – FUNGSI MEMBACA FILE .txt
def read_txt(path):
    """
    Membaca isi file .txt dan mengembalikan string utuh.
    """
    with open(path, 'r', encoding='utf-8') as f:
        return f.read()

# STEP 3 – MEMBACA SELURUH TEKS REFERENSI DARI FILE .txt
paths = {
    "Tafsir_Ibnu_Katsir": "/content/drive/MyDrive/Tafsir Evaluation/references/ibnu_katsir.txt",
    "Tafsir_Qurtubi": "/content/drive/MyDrive/Tafsir Evaluation/references/qurtubi.txt",
    "Tafsir_Thabari": "/content/drive/MyDrive/Tafsir Evaluation/references/thabari.txt"
}

references = {}
for key, path in paths.items():
    full_text = read_txt(path)
    references[key] = full_text
    print(f"✅ Referensi {key} berhasil dibaca. Jumlah karakter: {len(full_text)}")


# STEP 4 – PROMPT TEMPLATE DENGAN CONTOH
from langchain.prompts import PromptTemplate

# Baca isi prompt dari file eksternal
rubrik_mini_path = "/content/drive/MyDrive/Tafsir Evaluation/references/rubrik_mini.txt"
with open(rubrik_mini_path, "r", encoding="utf-8") as f:
    rubrik_full_text = f.read()

# Bangun PromptTemplate menggunakan isi file txt
rubrik_prompt = PromptTemplate(
    input_variables=["claim", "reference", "ayat"],
    template=rubrik_full_text
)

# STEP 5 – INISIALISASI MODEL DAN EVALUASI MENGGUNAKAN LANGCHAIN
# Gunakan model GPT-4o-mini sesuai keinginan dan naikkan max_tokens jika diperlukan, misalnya ke 1200 token
llm = ChatOpenAI(model_name="gpt-4", temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])

# Versi baru – REKOMENDASI LangChain ✅
evaluation_chain_runnable = rubrik_prompt | llm | output_parser

# Fungsi untuk mengonversi teks klaim (AI Response) ke format YAML
def convert_to_yaml(text):
    """Mengonversi teks ke format YAML agar LLM lebih mudah membaca struktur teks."""
    return yaml.safe_dump({"AI_Response": text}, allow_unicode=True)

# STEP 6–7: EVALUASI OTOMATIS DARI EXCEL
# Load data dari Excel; klaim diambil dari kolom "AI Response" dan ayat dari kolom "Ayat"
df = pd.read_excel("/content/drive/MyDrive/Tafsir Evaluation/chaining/DeepSeek_V2.xlsx")

# Definisikan parser untuk mengkonversi output evaluasi ke JSON
output_parser = StrOutputParser()

# Bangun rantai evaluasi sebagai Runnable (menggabungkan prompt, LLM, dan parser)
evaluation_chain_runnable: Runnable = rubrik_prompt | llm | output_parser

# STEP 6 – EVALUASI PER KITAB
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Konfigurasi chunker
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,  # maksimal karakter per chunk
    chunk_overlap=100
)

# Fungsi evaluasi per kitab dengan chunking
def evaluate_with_chunking(claim, ayat, tafsir_key, tafsir_text):
    chunks = splitter.split_text(tafsir_text)
    best_result = None
    best_score = 10  # konservatif: cari skor A terendah

    for i, chunk in enumerate(chunks):
        try:
            result = evaluation_chain_runnable.invoke({
                "claim": claim,
                "reference": chunk,
                "ayat": ayat
            })
            json_data = extract_json_from_response(result)
            score = json_data.get("A") if json_data else 10
            if score < best_score:
                best_score = score
                best_result = result
        except Exception as e:
            print(f"[ERROR CHUNK {i} | {tafsir_key}]: {e}")
            continue

    return best_result

# Simpan hasil
results_thabari, results_katsir, results_qurtubi = [], [], []

for idx, row in df.iterrows():
    raw_claim = row["AI Response"]
    ayat = row["Ayat"] if "Ayat" in row else "QS 9:60"

    print(f"🔍 Index {idx} | Mulai evaluasi...")

    try:
        for tafsir_key, storage in zip(
            ["Tafsir_Thabari", "Tafsir_Ibnu_Katsir", "Tafsir_Qurtubi"],
            [results_thabari, results_katsir, results_qurtubi]
        ):
            ref_text = references[tafsir_key]
            result = evaluate_with_chunking(raw_claim, ayat, tafsir_key, ref_text)
            print(f"✅ Index {idx} | {tafsir_key} OK")
            storage.append(result)
    except Exception as e:
        print(f"[❌ ERROR TOTAL @ {idx}]: {e}")
        results_thabari.append(None)
        results_katsir.append(None)
        results_qurtubi.append(None)

# Ekstrak skor JSON dari respons model
def extract_json_from_response(text):
    try:
        match = re.search(r"Final JSON:\s*(\{[\s\S]*?\})", text)
        if not match:
            print("[JSON WARNING]: Tidak ditemukan blok 'Final JSON'")
            return None
        json_str = match.group(1).strip()
        json_str = re.sub(r",\s*}", "}", json_str)
        json_str = re.sub(r",\s*]", "]", json_str)
        try:
            return ast.literal_eval(json_str)
        except:
            json_str = json_str.replace("‘", "'").replace("’", "'")
            json_str = json_str.replace("“", '"').replace("”", '"').replace("'", '"')
            return json.loads(json_str)
    except Exception as e:
        print(f"[JSON ERROR]: {e}")
        return None

# ✅ Ekstrak komentar naratif per label (A, B, C) – versi yang diperbaiki
def extract_comment(text, label):
    try:
        # Lebih fleksibel terhadap newline atau spasi setelah label
        pattern = rf"\[{label}\]\s*\n?(.*?)(?:Skor:|Score:)\s*\d+"
        match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
        if match:
            return match.group(1).strip()
    except Exception as e:
        print(f"[ERROR ekstrak {label}]: {e}")
    return ""

# STEP 8 – EKSTRAK SEMUA SKOR & KOMENTAR PER KITAB
def extract_all_scores(result_list, prefix):
    scores_A, scores_B, scores_C = [], [], []
    comments_A, comments_B, comments_C = [], [], []

    for idx, res in enumerate(result_list):
        if isinstance(res, str):
            json_data = extract_json_from_response(res)
            if json_data is None:
                print(f"[⛔️ JSON gagal @ index {idx} | {prefix}]")
                print(res[:300])
            scores_A.append(json_data.get("A") if json_data else None)
            scores_B.append(json_data.get("B") if json_data else None)
            scores_C.append(json_data.get("C") if json_data else None)
            comments_A.append(extract_comment(res, "A"))
            comments_B.append(extract_comment(res, "B"))
            comments_C.append(extract_comment(res, "C"))

            # ✅ Cetak evaluasi per dimensi langsung ke output
            print(f"\n🔎 EVALUASI {prefix} – Index {idx}")
            print(f"[A] {comments_A[-1]} | Skor: {scores_A[-1]}")
            print(f"[B] {comments_B[-1]} | Skor: {scores_B[-1]}")
            print(f"[C] {comments_C[-1]} | Skor: {scores_C[-1]}")
        else:
            scores_A.append(None)
            scores_B.append(None)
            scores_C.append(None)
            comments_A.append("")
            comments_B.append("")
            comments_C.append("")

    df[f"{prefix}_Validity"] = scores_A
    df[f"{prefix}_BilMa'tsur"] = scores_B
    df[f"{prefix}_Halusinasi"] = scores_C
    df[f"{prefix}_Komentar_A"] = comments_A
    df[f"{prefix}_Komentar_B"] = comments_B
    df[f"{prefix}_Komentar_C"] = comments_C

# STEP 9 – SIMPAN KE EXCEL
output_path = "/content/drive/MyDrive/Tafsir Evaluation/outputs/DeepSeek_Evaluated_PerTafsir_OpenAI_V2.xlsx"
extract_all_scores(results_thabari, "Thabari")
extract_all_scores(results_katsir, "Ibnu_Katsir")
extract_all_scores(results_qurtubi, "Qurtubi")
df.to_excel(output_path, index=False)
print(f"✅ Evaluasi selesai! Hasil disimpan di: {output_path}")

Masukkan OpenAI API key Anda: ··········
✅ Referensi Tafsir_Ibnu_Katsir berhasil dibaca. Jumlah karakter: 2542
✅ Referensi Tafsir_Qurtubi berhasil dibaca. Jumlah karakter: 6796
✅ Referensi Tafsir_Thabari berhasil dibaca. Jumlah karakter: 4362
🔍 Index 0 | Mulai evaluasi...
✅ Index 0 | Tafsir_Thabari OK
✅ Index 0 | Tafsir_Ibnu_Katsir OK
✅ Index 0 | Tafsir_Qurtubi OK
🔍 Index 1 | Mulai evaluasi...
✅ Index 1 | Tafsir_Thabari OK
✅ Index 1 | Tafsir_Ibnu_Katsir OK
✅ Index 1 | Tafsir_Qurtubi OK
🔍 Index 2 | Mulai evaluasi...
✅ Index 2 | Tafsir_Thabari OK
✅ Index 2 | Tafsir_Ibnu_Katsir OK
✅ Index 2 | Tafsir_Qurtubi OK
🔍 Index 3 | Mulai evaluasi...
✅ Index 3 | Tafsir_Thabari OK
✅ Index 3 | Tafsir_Ibnu_Katsir OK
✅ Index 3 | Tafsir_Qurtubi OK

🔎 EVALUASI Thabari – Index 0
[A] Klaim ini didukung oleh kutipan dari Al-Ṭabarī, Ibn Kathīr, dan Al-Qurṭubī. Klaim tersebut merujuk kepada tafsir dari para sahabat dan tabi'in seperti Ibn ʿAbbās, Al-Ḥasan al-Baṣrī, Ibn Zayd, dan Qatādah yang disampaikan oleh A

In [ ]:
import pandas as pd

# Load hasil evaluasi
df_path = "/content/drive/MyDrive/Tafsir Evaluation/outputs/DeepSeek_Evaluated_PerTafsir_OpenAI_V2.xlsx"
df = pd.read_excel(df_path)

# Tampilkan 10 baris pertama untuk preview
pd.set_option('display.max_columns', None)  # agar semua kolom terlihat
pd.set_option('display.max_colwidth', 150)  # agar isi kolom komentar tidak terpotong
df.head(10)


,Platfrom,Prompt Technique,AI Response,Tafsir_Validity,Tafir Bil Ma'tsur Methods,Halucination,Tafsir_Validity_Score,Bil_Matsur_Methods_Score,Halucination_score,Thabari_Validity,Thabari_BilMa'tsur,Thabari_Halusinasi,Thabari_Komentar_A,Thabari_Komentar_B,Thabari_Komentar_C,Ibnu_Katsir_Validity,Ibnu_Katsir_BilMa'tsur,Ibnu_Katsir_Halusinasi,Ibnu_Katsir_Komentar_A,Ibnu_Katsir_Komentar_B,Ibnu_Katsir_Komentar_C,Qurtubi_Validity,Qurtubi_BilMa'tsur,Qurtubi_Halusinasi,Qurtubi_Komentar_A,Qurtubi_Komentar_B,Qurtubi_Komentar_C
0,DeepSeek,Zero Shot,"In Surah At-Tawbah (9:60), the category ""fī sabīlillāh"" (في سبيل الله) is one of the eight recipients of zakāt. Classical exegetes, relying on the...","Ibn ʿAbbās is indeed mentioned by al-Ṭabarī as one of those who interpreted ""fī sabīlillāh"" to mean fighters in jihad. However, there is no explic...","Weakness: The response does not include explicit citations or chains of transmission (sanad). Although it mentions figures from the salaf, the nar...","Extrinsic Hallucination: It is claimed that Ibn Kathīr cited Mujāhid, Ibn ʿUmar, and ʿAṭāʾ ibn Abī Rabāḥ. However, there is no explicit mention in...",4,7,4,7,7,10,"Klaim ini didukung oleh kutipan dari Al-Ṭabarī, Ibn Kathīr, dan Al-Qurṭubī. Klaim tersebut merujuk kepada tafsir dari para sahabat dan tabi'in sep...","Klaim ini mengikuti struktur tafsir bil ma'tsur dengan merujuk kepada ayat, riwayat, dan mufasir. Namun, klaim ini tidak menyertakan sanad atau ku...","Klaim ini tidak mengandung halusinasi tafsir. Semua klaim, tokoh, dan hadith yang disebutkan didukung oleh sumber.",6,5,4,NaN,NaN,NaN,4,5,4,"Klaim ini merujuk kepada tafsir dari Al-Ṭabarī, Ibn Kathīr, dan Al-Qurṭubī. Klaim tersebut mencakup berbagai pendapat dari para sahabat dan tabi'i...","Klaim ini tidak mengikuti struktur klasik ayat → riwayat → mufasir. Meskipun klaim tersebut merujuk kepada ayat dan mufasir, tidak ada riwayat ata...","Klaim ini mengandung beberapa halusinasi. Pertama, klaim tersebut merujuk kepada pendapat Ibn ʿAbbās, Al-Ḥasan al-Baṣrī, Ibn Zayd, dan Qatādah, te..."
1,DeepSeek,Few Shot,"In Jāmiʿ al-Bayān ʿan Taʾwīl Āy al-Qurʾān, Imām al-Ṭabarī explains that the term fī sabīlillāh in Surah At-Tawbah (9:60) primarily denotes those e...",There is no explicit quotation from Ibn ʿAbbās in the available text stating that fī sabīlillāh refers to “those who go out in the path of Allah w...,There is no literal citation of the verse or narrations as the basis for the interpretation. The structure does not follow the disciplined sequenc...,Hallucination or Unproven Claims: The statement “Ibn ʿAbbās said: those who go out in the path of Allah without receiving a salary” is not found i...,2,5,4,4,5,4,"Klaim ini merujuk kepada tafsir dari Imam al-Ṭabarī, Imam al-Qurṭubī, dan Ibn Kathīr. Namun, dalam referensi yang diberikan, hanya kutipan dari Ta...","Klaim ini tidak mengikuti struktur bil ma'tsur. Meskipun klaim tersebut merujuk kepada beberapa mufasir dan hadith, tidak ada ayat yang disebutkan...","Klaim ini mengandung beberapa halusinasi. Klaim tersebut merujuk kepada pendapat dan hadith dari beberapa tokoh, namun tidak ada referensi untuk i...",6,5,4,"Klaim ini didukung oleh kutipan dari Tafsir Ibnu Katsir. Klaim tersebut menyebutkan bahwa ""fī sabīlillāh"" merujuk kepada pejuang (ghuzāt) yang tid...","Klaim ini mengikuti struktur bil ma'tsur dengan menyebutkan ayat, riwayat, dan mufasir. Namun, klaim tersebut tidak mencantumkan sanad atau kutipa...","Klaim ini mengandung beberapa halusinasi. Meskipun sebagian besar klaim didukung oleh kutipan yang sahih, ada beberapa klaim yang tidak didukung s...",4,4,3,"Klaim ini merujuk kepada tafsir dari Imām al-Ṭabarī, Imam al-Qurṭubī, dan Ibn Kathīr. Klaim tersebut menyatakan bahwa ""fī sabīlillāh"" dalam Surah ...",Klaim ini tidak mengikuti struktur klasik ayat → riwayat → mufasir. Klaim ini hanya merujuk kepada mufasir dan pendapat mereka tanpa memberikan ay...,"Klaim ini mengandung beberapa halusinasi. Misalnya, klaim tersebut m

In [ ]:
import pandas as pd
import numpy as np

# STEP 1 – Load DataFrame
df_path = "/content/drive/MyDrive/Tafsir Evaluation/outputs/DeepSeek_Evaluated_PerTafsir_OpenAI_V2.xlsx"
df = pd.read_excel(df_path)

# STEP 2 – Ambil skor manual
manual_validity = df["Tafsir_Validity_Score"]
manual_bilma = df["Bil_Matsur_Methods_Score"]
manual_hallu = df["Halucination_score"]

# STEP 3 – Ambil skor otomatis per dimensi
auto_validity_opsi = df[["Thabari_Validity", "Ibnu_Katsir_Validity", "Qurtubi_Validity"]]
auto_bilma_opsi = df[["Thabari_BilMa'tsur", "Ibnu_Katsir_BilMa'tsur", "Qurtubi_BilMa'tsur"]]
auto_hallu_opsi = df[["Thabari_Halusinasi", "Ibnu_Katsir_Halusinasi", "Qurtubi_Halusinasi"]]

# STEP 4 – Fungsi pemilih skor otomatis terdekat
def pilih_skor_terdekat(manual_series, opsi_df):
    skor_terdekat = []
    for i in range(len(manual_series)):
        manual = manual_series.iloc[i]
        opsi = opsi_df.iloc[i]
        if pd.isna(manual):
            skor_terdekat.append(np.nan)
        else:
            skor_terdekat.append(opsi.iloc[(opsi - manual).abs().argmin()])
    return skor_terdekat

# STEP 5 – Simpan skor otomatis terdekat ke kolom baru
df["Tafsir_Validity_Score_Auto"] = pilih_skor_terdekat(manual_validity, auto_validity_opsi)
df["Bil_Matsur_Methods_Score_Auto"] = pilih_skor_terdekat(manual_bilma, auto_bilma_opsi)
df["Halucination_Score_Auto"] = pilih_skor_terdekat(manual_hallu, auto_hallu_opsi)

# STEP 6 – Hitung akurasi dan MAE
def hitung_akurasi_mae(manual, pred):
    exact_match = (manual == pred).mean()
    toleransi_1 = ((manual - pred).abs() <= 1).mean()
    mae = (manual - pred).abs().mean()
    return exact_match, toleransi_1, mae

acc_v, tol_v, mae_v = hitung_akurasi_mae(manual_validity, df["Tafsir_Validity_Score_Auto"])
acc_b, tol_b, mae_b = hitung_akurasi_mae(manual_bilma, df["Bil_Matsur_Methods_Score_Auto"])
acc_h, tol_h, mae_h = hitung_akurasi_mae(manual_hallu, df["Halucination_Score_Auto"])

# STEP 7 – Cetak laporan
print("📊 AKURASI & MAE:")
print(f"[VALIDITAS]     🎯 Exact: {acc_v:.2%} | ±1: {tol_v:.2%} | MAE: {mae_v:.2f}")
print(f"[BIL MA'TSUR]   📘 Exact: {acc_b:.2%} | ±1: {tol_b:.2%} | MAE: {mae_b:.2f}")
print(f"[HALUSINASI]    🔴 Exact: {acc_h:.2%} | ±1: {tol_h:.2%} | MAE: {mae_h:.2f}")

# STEP 8 – Simpan hasil
output_path = "/content/drive/MyDrive/Tafsir Evaluation/outputs/DeepSeek_Evaluated_WithAuto_V2.xlsx"
df.to_excel(output_path, index=False)
print(f"✅ File disimpan ke: {output_path}")


📊 AKURASI & MAE:
[VALIDITAS]     🎯 Exact: 50.00% | ±1: 75.00% | MAE: 0.75
[BIL MA'TSUR]   📘 Exact: 50.00% | ±1: 100.00% | MAE: 0.50
[HALUSINASI]    🔴 Exact: 100.00% | ±1: 100.00% | MAE: 0.00
✅ File disimpan ke: /content/drive/MyDrive/Tafsir Evaluation/outputs/DeepSeek_Evaluated_WithAuto_V2.xlsx


In [ ]:
# ✅ KODE LENGKAP UNTUK SIMULASI 1 RESPON (TIDAK DIREDUKSI)

# STEP 1 – IMPORT & SETUP
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable
import yaml
from docx import Document
import getpass
import pandas as pd
import json
import os
import time
import re
import ast
from langchain.text_splitter import RecursiveCharacterTextSplitter

# SET API KEY
os.environ["OPENAI_API_KEY"] = getpass.getpass("Masukkan OpenAI API key Anda: ")

# STEP 2 – Fungsi Baca File .txt

def read_txt(path):
    with open(path, 'r', encoding='utf-8') as f:
        return f.read()

# STEP 3 – Load Referensi Tafsir
paths = {
    "Tafsir_Ibnu_Katsir": "/content/drive/MyDrive/Tafsir Evaluation/references/ibnu_katsir.txt",
    "Tafsir_Qurtubi": "/content/drive/MyDrive/Tafsir Evaluation/references/qurtubi.txt",
    "Tafsir_Thabari": "/content/drive/MyDrive/Tafsir Evaluation/references/thabari.txt"
}

references = {}
for key, path in paths.items():
    full_text = read_txt(path)
    references[key] = full_text
    print(f"✅ Referensi {key} berhasil dibaca. Jumlah karakter: {len(full_text)}")

# STEP 4 – PromptTemplate dari File Eksternal
rubrik_path = "/content/drive/MyDrive/Tafsir Evaluation/references/rubrik_mini.txt"
with open(rubrik_path, "r", encoding="utf-8") as f:
    rubrik_full_text = f.read()

rubrik_prompt = PromptTemplate(
    input_variables=["claim", "reference", "ayat"],
    template=rubrik_full_text
)

# STEP 5 – Inisialisasi Model
llm = ChatOpenAI(model_name="gpt-4", temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])
output_parser = StrOutputParser()
evaluation_chain_runnable: Runnable = rubrik_prompt | llm | output_parser

# STEP 6 – Text Splitter untuk Chunking
splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)

# STEP 7 – Ekstraksi JSON dan Komentar

def extract_json_from_response(text):
    try:
        match = re.search(r"Final JSON:\s*(\{[\s\S]*?\})", text)
        if not match:
            return None
        json_str = match.group(1).strip()
        json_str = re.sub(r",\s*\}", "}", json_str)
        json_str = re.sub(r",\s*\]", "]", json_str)
        try:
            return ast.literal_eval(json_str)
        except:
            json_str = json_str.replace("‘", "'").replace("’", "'").replace("“", '"').replace("”", '"')
            json_str = json_str.replace("'", '"')
            return json.loads(json_str)
    except:
        return None

def extract_comment(text, label):
    pattern = rf"\[{label}\]\s*\n?(.*?)(?:Skor:|Score:)\s*\d+"
    match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return ""

# STEP 8 – Fungsi Evaluasi Manual

def uji_respon_manual(claim, ayat="QS 9:60", kitab="Tafsir_Thabari"):
    text_chunks = splitter.split_text(references[kitab])
    best_result = None
    best_score = 10

    for i, chunk in enumerate(text_chunks):
        try:
            result = evaluation_chain_runnable.invoke({
                "claim": claim,
                "reference": chunk,
                "ayat": ayat
            })
            json_data = extract_json_from_response(result)
            score = json_data.get("A") if json_data else 10
            if score < best_score:
                best_score = score
                best_result = result
        except Exception as e:
            print(f"[ERROR @ chunk {i}] {e}")

    if best_result:
        print("\n📌 HASIL EVALUASI TERPILIH:")
        print(best_result)
        print("\n🔢 SKOR FINAL:")
        print(extract_json_from_response(best_result))
        print("\n📝 KOMENTAR:")
        print(f"[A] {extract_comment(best_result, 'A')}")
        print(f"[B] {extract_comment(best_result, 'B')}")
        print(f"[C] {extract_comment(best_result, 'C')}")
    else:
        print("⚠️ Tidak ada hasil evaluasi yang valid ditemukan.")


Masukkan OpenAI API key Anda: ··········
✅ Referensi Tafsir_Ibnu_Katsir berhasil dibaca. Jumlah karakter: 2542
✅ Referensi Tafsir_Qurtubi berhasil dibaca. Jumlah karakter: 6796
✅ Referensi Tafsir_Thabari berhasil dibaca. Jumlah karakter: 4362


In [ ]:
# Masukkan teks respons AI yang ingin dievaluasi di sini
respon_anda = """
Al-Tabari (d. 923 CE), in his Jami‘ al-Bayan fi Ta’wil Ay al-Qur’an, is known for his exhaustive compilation of early scholarly opinions, often presenting multiple interpretations with their chains of narration (isnads) before offering his preferred view. His approach to "fi sabilillah" is similarly detailed, reflecting the diversity of early Islamic thought.

In Surah Al-Baqarah (2:154), which mentions those "killed in the way of Allah" (قُتِلُوا فِي سَبِيلِ اللَّهِ), Al-Tabari records various opinions from the Salaf (early generations). Some Companions, like Ibn Abbas, reportedly said it specifically means those who die fighting disbelievers to uphold Allah’s religion. Others extended it to include any death while performing a righteous act, such as dying during pilgrimage or in pursuit of knowledge, though Al-Tabari favors the narrower interpretation tied to martyrdom in battle, arguing it aligns with the verse’s context of encouraging steadfastness against enemies. He emphasizes that "fi sabilillah" denotes an intention purely for Allah, excluding worldly motives, and supports this with linguistic analysis of "sabil" (path) as the path of obedience to Allah.
"""

# Pilih ayat dan kitab tafsir
ayat = "QS 9:60"
kitab = "Tafsir_Thabari"  # atau: "Tafsir_Ibnu_Katsir" / "Tafsir_Qurtubi"

# Jalankan evaluasi (gunakan chunking per kitab)
ref_text = references[kitab]
result = evaluate_with_chunking(respon_anda, ayat, kitab, ref_text)

# Cetak hasil
print(f"📘 Evaluasi {kitab} untuk ayat {ayat}:\n\n{result}")

# Ekstrak skor
json_data = extract_json_from_response(result)
if json_data:
    print("\n📊 Skor Evaluasi:")
    print(f"A – Validitas Tafsir     : {json_data.get('A')}")
    print(f"B – Bil Ma'tsūr Structure: {json_data.get('B')}")
    print(f"C – Halusinasi Tafsir    : {json_data.get('C')}")

    # Ekstrak komentar naratif
    print("\n📝 Komentar:")
    for label in ["A", "B", "C"]:
        komentar = extract_comment(result, label)
        print(f"[{label}] {komentar}")


📘 Evaluasi Tafsir_Thabari untuk ayat QS 9:60:

[A]
Klaim ini tidak didukung oleh kutipan dari Tafsir Al-Tabari. Klaim tersebut merujuk pada Surah Al-Baqarah (2:154), bukan QS 9:60 yang seharusnya menjadi fokus. Selain itu, klaim tersebut juga merujuk pada beberapa pendapat dari Salaf dan interpretasi Al-Tabari tentang "fi sabilillah", namun tidak ada kutipan langsung atau rujukan spesifik yang mendukung klaim tersebut dalam teks referensi. Oleh karena itu, klaim ini tidak valid dan tidak didukung oleh kutipan yang sahih.
Skor: 1

[B]
Klaim ini tidak mengikuti struktur bil ma'tsur. Meskipun klaim tersebut merujuk pada beberapa mufasir dan pendapat mereka, tidak ada ayat, riwayat, atau isnad yang disebutkan secara eksplisit. Selain itu, klaim tersebut merujuk pada Surah Al-Baqarah (2:154), bukan QS 9:60 yang seharusnya menjadi fokus. Oleh karena itu, struktur klaim ini tidak sesuai dengan metode bil ma'tsur.
Skor: 1

[C]
Klaim ini mengandung beberapa halusinasi. Pertama, klaim tersebut m